<a href="https://colab.research.google.com/github/calde97/Deep_Learning_Challenge/blob/main/second_challenge/Unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unet

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


## Create Dataset

Remember you should move all your images under images and masks under masks.

Directory structure :

    - challenge2/DS/ 
                 - Images/
                     - img1, img2, …, imgN 
                 - Masks/
                     - img1, img2, …, imgN      
                 - Splits
                     - train.txt, val.txt


In [ ]:
%cd /content/drive/MyDrive/challenge2/DS

In [ ]:
#The following function creates two files that contain the names of the training and validation images

def write_training(images, masks, split=0.15, seed=1234, training_file_name='train.txt', validation_file_name='val.txt'):
  np.random.seed(seed)
  np.random.shuffle(images)
  validation_split = 0.15
  separation_index = int(split * len(images)) #Get the n-element for the splitting

  #Separating the names for training and validation
  validation_images = images[0:separation_index]
  training_images = images[separation_index:]
  
  #Write the training names
  with open('Splits/' + training_file_name, mode='w') as f:
    for image_name in training_images:
      image_name = image_name.split('.')[0] #Removing the .jpg postfix
      f.write(image_name + '\n')

  #Write the validation names
  with open('Splits/' + validation_file_name, mode='w') as f:
    for image_name in validation_images:
      image_name = image_name.split('.')[0] #Removing the .jpg postfix
      f.write(image_name + '\n')


Take all the image and mask names in an array

In [ ]:
images = os.listdir('Images')
masks = os.listdir('Masks')
images.sort()
masks.sort()

Creates the files under splits that contain the names of the training and validation images

In [ ]:
write_training(images, masks, split=0.15, seed=1234, training_file_name='train.txt', validation_file_name='val.txt')

Create the generators


In [ ]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
cwd = os.getcwd()
image_path = os.path.join(cwd, 'Images')
mask_path = os.path.join(cwd, 'Masks')

validation_split=0.15
bs = 2
img_height = 768
img_width = 1024
seed = 1234

#Remember to delete rescale if using a different preprocess function
train_image_data_generator = ImageDataGenerator(rescale=1/255.,
                                                validation_split=validation_split,
                                                width_shift_range=0.05,
                                                height_shift_range=0.05,
                                                rotation_range=5,
                                                zoom_range=0.2,
                                                horizontal_flip=True,
                                                fill_mode='reflect')

train_mask_image_data_generator = ImageDataGenerator(width_shift_range=0.05,
                                                validation_split=validation_split,
                                                height_shift_range=0.05,
                                                rotation_range=5,
                                                zoom_range=0.2,
                                                horizontal_flip=True,
                                                fill_mode='reflect')

Define our custom dataset 

In [ ]:
from PIL import Image

class CustomDataset(tf.keras.utils.Sequence):

  """
    CustomDataset inheriting from tf.keras.utils.Sequence.

    3 main methods:
      - __init__: save dataset params like directory, filenames..
      - __len__: return the total number of samples in the dataset
      - __getitem__: return a sample from the dataset

    Note: 
      - the custom dataset return a single sample from the dataset. Then, we use 
        a tf.data.Dataset object to group samples into batches.
      - in this case we have a different structure of the dataset in memory. 
        We have all the images in the same folder and the training and validation splits
        are defined in text files.

  """

  def __init__(self, dataset_dir, which_subset, img_generator=None, mask_generator=None, 
               preprocessing_function=None, out_shape=[256, 256]):
    if which_subset == 'training':
      subset_file = os.path.join(dataset_dir, 'Splits', 'train.txt')
    elif which_subset == 'validation':
      subset_file = os.path.join(dataset_dir, 'Splits', 'val.txt')
    
    with open(subset_file, 'r') as f:
      lines = f.readlines()
    
    subset_filenames = []
    for line in lines:
      subset_filenames.append(line.strip()) 

    self.which_subset = which_subset
    self.dataset_dir = dataset_dir
    self.subset_filenames = subset_filenames
    self.img_generator = img_generator
    self.mask_generator = mask_generator
    self.preprocessing_function = preprocessing_function
    self.out_shape = out_shape

  def __len__(self):
    return len(self.subset_filenames)

  def __getitem__(self, index):
    # Read Image
    curr_filename = self.subset_filenames[index]
    if (curr_filename[0] == 'R'):
      img = Image.open(os.path.join(self.dataset_dir, 'Images', curr_filename + '.png'))
    else:
      img = Image.open(os.path.join(self.dataset_dir, 'Images', curr_filename + '.jpg'))
      
    mask = Image.open(os.path.join(self.dataset_dir, 'Masks', curr_filename + '.png'))
  
    # Resize image and mask
    img = img.resize(self.out_shape)
    mask = mask.resize(self.out_shape, resample=Image.NEAREST)
    
    img_arr = np.array(img)
    mask_arr = np.array(mask)

    # in this dataset 255 mask label is assigned to an additional class, which corresponds 
    # to the contours of the objects. We remove it for simplicity.
    #mask_arr[mask_arr == 255] = 0  

    ############################################################################
    #Part added. We have to read the mask as provided by the script.
    new_mask_arr = np.zeros(mask_arr.shape[:2], dtype=mask_arr.dtype)
    # Use RGB dictionary in 'RGBtoTarget.txt' to convert RGB to target
    new_mask_arr[np.where(np.all(mask_arr == [216, 124, 18], axis=-1))] = 0
    new_mask_arr[np.where(np.all(mask_arr == [255, 255, 255], axis=-1))] = 1
    new_mask_arr[np.where(np.all(mask_arr == [216, 67, 82], axis=-1))] = 2
    ############################################################################

    mask_arr = np.expand_dims(new_mask_arr, -1) #Modified

    if self.which_subset == 'training':
      if self.img_generator is not None and self.mask_generator is not None:
        # Perform data augmentation
        # We can get a random transformation from the ImageDataGenerator using get_random_transform
        # and we can apply it to the image using apply_transform
        img_t = self.img_generator.get_random_transform(img_arr.shape, seed=seed)
        mask_t = self.mask_generator.get_random_transform(mask_arr.shape, seed=seed)
        img_arr = self.img_generator.apply_transform(img_arr, img_t)
        # ImageDataGenerator use bilinear interpolation for augmenting the images.
        # Thus, when applied to the masks it will output 'interpolated classes', which
        # is an unwanted behaviour. As a trick, we can transform each class mask 
        # separately and then we can cast to integer values (as in the binary segmentation notebook).
        # Finally, we merge the augmented binary masks to obtain the final segmentation mask.
        out_mask = np.zeros_like(mask_arr)
        for c in np.unique(mask_arr):
          if c > 0:
            curr_class_arr = np.float32(mask_arr == c)
            curr_class_arr = self.mask_generator.apply_transform(curr_class_arr, mask_t)
            # from [0, 1] to {0, 1}
            curr_class_arr = np.uint8(curr_class_arr)
            # recover original class
            curr_class_arr = curr_class_arr * c 
            out_mask += curr_class_arr
    else:
      out_mask = mask_arr
    
    if self.preprocessing_function is not None:
        img_arr = self.preprocessing_function(img_arr)

    return img_arr, np.float32(out_mask)

In [ ]:
def normalize(img):
  return img / 255.

Create the train and validation dataset

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input 


dataset = CustomDataset('/content/drive/MyDrive/challenge2/DS', 'training', 
                        img_generator=train_image_data_generator, mask_generator=train_mask_image_data_generator,
                        preprocessing_function=normalize, out_shape=[img_width, img_height]) #HERE the shape is inverted => BAD thing but is was already written by the prof

dataset_valid = CustomDataset('/content/drive/MyDrive/challenge2/DS', 'validation', 
                        img_generator=train_image_data_generator, mask_generator=train_mask_image_data_generator,
                        preprocessing_function=normalize, out_shape=[img_width, img_height]) #HERE the shape is inverted => BAD thing but is was already written by the prof

In [ ]:
#Train dataset
train_dataset = tf.data.Dataset.from_generator(lambda: dataset,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([img_height, img_width, 3], [img_height, img_width, 1]))
 
train_dataset = train_dataset.batch(bs)
 
train_dataset = train_dataset.repeat()

##Validation dataset

validation_dataset = tf.data.Dataset.from_generator(lambda: dataset_valid,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([img_height, img_width, 3], [img_height, img_width, 1]))
 
validation_dataset = validation_dataset.batch(bs)
 
validation_dataset = validation_dataset.repeat()

## Model architecture

The architecture is a classical Unet architecture

In [ ]:
import tensorflow as tf
def UNet(input_size = (896,896,3)):
    inputs = tf.keras.layers.Input(input_size)
    conv1 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    conv1=tf.keras.layers.BatchNormalization()(conv1)
    pool1 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    conv2=tf.keras.layers.BatchNormalization()(conv2)
    pool2 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    conv3=tf.keras.layers.BatchNormalization()(conv3)
    pool3 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    conv4=tf.keras.layers.BatchNormalization()(conv4)
    pool4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)
    conv5 = tf.keras.layers.Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = tf.keras.layers.Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    conv5=tf.keras.layers.BatchNormalization()(conv5)
    drop5 = tf.keras.layers.Dropout(0.5)(conv5)
    up6 = tf.keras.layers.Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(tf.keras.layers.UpSampling2D(size = (2,2))(drop5))
    merge6 = tf.keras.layers.concatenate([conv4,up6], axis = 3)
    conv6 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = tf.keras.layers.Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)
    conv6=  tf.keras.layers.BatchNormalization()(conv6)
    up7 = tf.keras.layers.Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(tf.keras.layers.UpSampling2D(size = (2,2))(conv6))
    merge7 = tf.keras.layers.concatenate([conv3,up7], axis = 3)
    conv7 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = tf.keras.layers.Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)
    conv7=tf.keras.layers.BatchNormalization()(conv7)
    up8 = tf.keras.layers.Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(tf.keras.layers.UpSampling2D(size = (2,2))(conv7))
    merge8 = tf.keras.layers.concatenate([conv2,up8], axis = 3)
    conv8 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = tf.keras.layers.Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)
    conv8=tf.keras.layers.BatchNormalization()(conv8)
    up9 = tf.keras.layers.Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(tf.keras.layers.UpSampling2D(size = (2,2))(conv8))
    merge9 = tf.keras.layers.concatenate([conv1,up9], axis = 3)
    conv9 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = tf.keras.layers.Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 =tf.keras.layers.Conv2D(3, 1, activation = 'softmax')(conv9)
    model = tf.keras.models.Model(inputs = inputs, outputs = conv10)
    return model

In [ ]:
model = UNet(input_size=(768, 1024, 3))
#model.summary()

## Model compiling

In [ ]:
def meanIoU(y_true, y_pred):
    # get predicted class from softmax
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)

    per_class_iou = []

    for i in range(1,3): # exclude the background class 0
      # Get prediction and target related to only a single class (i)
      class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
      class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
      intersection = tf.reduce_sum(class_true * class_pred)
      union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
      iou = (intersection + 1e-7) / (union + 1e-7)
      per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)

In [ ]:
def IoUCrop(y_true, y_pred):
    # get predicted class from softmax
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)

    per_class_iou = []

    for i in range(1,2): # exclude the background class 0
      # Get prediction and target related to only a single class (i)
      class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
      class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
      intersection = tf.reduce_sum(class_true * class_pred)
      union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
      iou = (intersection + 1e-7) / (union + 1e-7)
      per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)

In [ ]:
def IoUWeed(y_true, y_pred):
    # get predicted class from softmax
    y_pred = tf.expand_dims(tf.argmax(y_pred, -1), -1)

    per_class_iou = []

    for i in range(2,3): # exclude the background class 0
      # Get prediction and target related to only a single class (i)
      class_pred = tf.cast(tf.where(y_pred == i, 1, 0), tf.float32)
      class_true = tf.cast(tf.where(y_true == i, 1, 0), tf.float32)
      intersection = tf.reduce_sum(class_true * class_pred)
      union = tf.reduce_sum(class_true) + tf.reduce_sum(class_pred) - intersection
    
      iou = (intersection + 1e-7) / (union + 1e-7)
      per_class_iou.append(iou)

    return tf.reduce_mean(per_class_iou)

In [ ]:
metrics = ['accuracy', meanIoU, IoUCrop, IoUWeed]
loss = tf.keras.losses.SparseCategoricalCrossentropy() 
# learning rate
lr = 1e-4
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/challenge2/DS/experiment_results_unet --port 6005

In [ ]:
import os
from datetime import datetime


exps_dir = 'experiment_results_unet'
if not os.path.exists(exps_dir):
    os.makedirs(exps_dir)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)
    
callbacks = []

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=True,
                                                   save_best_only=False)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
tb_dir = os.path.join(exp_dir, 'tb_logs')
if not os.path.exists(tb_dir):
    os.makedirs(tb_dir)
    
# By default shows losses and metrics for both training and validation
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
                                             profile_batch=0,
                                             histogram_freq=0)  # if 1 shows weights histograms
callbacks.append(tb_callback)

# Early Stopping
# --------------
early_stop = True
if early_stop:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)


# How to visualize Tensorboard

# 1. tensorboard --logdir EXPERIMENTS_DIR --port PORT     <- from terminal
# 2. localhost:PORT   <- in your browser

## Model fitting

There were 2 trainings due to a colab timeout.

In [ ]:
model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(dataset),
          validation_data=validation_dataset,
          validation_steps=len(dataset_valid), 
          callbacks=callbacks)

Epoch 1/100
612/612 [==============================] - 1515s 2s/step - loss: 0.0972 - accuracy: 0.9619 - meanIoU: 0.5677 - val_loss: 0.2124 - val_accuracy: 0.9068 - val_meanIoU: 0.4697
Epoch 2/100
612/612 [==============================] - 1255s 2s/step - loss: 0.0759 - accuracy: 0.9707 - meanIoU: 0.6552 - val_loss: 0.1501 - val_accuracy: 0.9413 - val_meanIoU: 0.5067
Epoch 3/100
612/612 [==============================] - 1242s 2s/step - loss: 0.0664 - accuracy: 0.9746 - meanIoU: 0.6991 - val_loss: 0.1395 - val_accuracy: 0.9547 - val_meanIoU: 0.5607
Epoch 4/100
612/612 [==============================] - 1209s 2s/step - loss: 0.0583 - accuracy: 0.9774 - meanIoU: 0.7315 - val_loss: 0.1011 - val_accuracy: 0.9643 - val_meanIoU: 0.6027
Epoch 5/100
612/612 [==============================] - 1245s 2s/step - loss: 0.0526 - accuracy: 0.9793 - meanIoU: 0.7538 - val_loss: 0.0961 - val_accuracy: 0.9663 - val_meanIoU: 0.5998
Epoch 6/100
612/612 [==============================] - 1236s 2s/step - loss

In [ ]:
model.fit(x=train_dataset,
          epochs=100,  #### set repeat in training dataset
          steps_per_epoch=len(dataset) / bs,
          validation_data=validation_dataset,
          validation_steps=len(dataset_valid) / bs, 
          callbacks=callbacks)

Epoch 1/100
612/612 [==============================] - 1915s 3s/step - loss: 0.0456 - accuracy: 0.9816 - meanIoU: 0.7820 - val_loss: 0.1080 - val_accuracy: 0.9654 - val_meanIoU: 0.6424
Epoch 2/100
612/612 [==============================] - 1205s 2s/step - loss: 0.0419 - accuracy: 0.9828 - meanIoU: 0.7964 - val_loss: 0.0933 - val_accuracy: 0.9699 - val_meanIoU: 0.6741
Epoch 3/100
612/612 [==============================] - 1185s 2s/step - loss: 0.0394 - accuracy: 0.9837 - meanIoU: 0.8064 - val_loss: 0.0912 - val_accuracy: 0.9710 - val_meanIoU: 0.6659
Epoch 4/100
612/612 [==============================] - 1209s 2s/step - loss: 0.0362 - accuracy: 0.9848 - meanIoU: 0.8186 - val_loss: 0.0946 - val_accuracy: 0.9709 - val_meanIoU: 0.6483
Epoch 5/100
612/612 [==============================] - 1239s 2s/step - loss: 0.0370 - accuracy: 0.9846 - meanIoU: 0.8157 - val_loss: 0.0996 - val_accuracy: 0.9679 - val_meanIoU: 0.6402
Epoch 6/100
 65/612 [==>...........................] - ETA: 16:34 - loss: 0

KeyboardInterrupt: ignored

## Compute predictions

Creating all the test generators for computing the predictions. 
The target size should be the one used during the model training for better performances.
All the paths correspond to the paths in which all the images are saved

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
seed = 1234
h = 768
w = 1024

test_image_data_generator = ImageDataGenerator(rescale=1/255.)
test = test_image_data_generator.flow_from_directory('/content/drive/MyDrive/challenge2/Development_Dataset/Test_Dev/Bipbip/Haricot' #path to test images
                                                     ,classes=None, batch_size=1,
                                                     shuffle=False,
                                                     target_size=(h,w),
                                                     interpolation='bilinear',
                                                     seed=seed)

test1 = test_image_data_generator.flow_from_directory('/content/drive/MyDrive/challenge2/Development_Dataset/Test_Dev/Bipbip/Mais',                                    
                                                      classes=None, batch_size=1,
                                                      shuffle=False,
                                                      target_size=(h,w),
                                                      interpolation='bilinear',
                                                      seed=seed)
test2 = test_image_data_generator.flow_from_directory(
                                                      '/content/drive/MyDrive/challenge2/Development_Dataset/Test_Dev/Pead/Haricot' ,                                   
                                                      classes=None, batch_size=1,
                                                      shuffle=False,
                                                      interpolation='bilinear',
                                                      target_size=(h,w),
                                                      seed=seed)
test3 = test_image_data_generator.flow_from_directory(
                                                      '/content/drive/MyDrive/challenge2/Development_Dataset/Test_Dev/Pead/Mais'     ,                               
                                                      classes=None, batch_size=1,
                                                      shuffle=False,
                                                      target_size=(h,w),
                                                      interpolation='bilinear',
                                                      seed=seed)
test4 = test_image_data_generator.flow_from_directory(
                                                      '/content/drive/MyDrive/challenge2/Development_Dataset/Test_Dev/Roseau/Haricot' ,                                   
                                                      classes=None, batch_size=1,
                                                      shuffle=False,
                                                      target_size=(h,w),
                                                      interpolation='bilinear',
                                                      seed=seed)
test5 = test_image_data_generator.flow_from_directory(
                                                      '/content/drive/MyDrive/challenge2/Development_Dataset/Test_Dev/Roseau/Mais'  ,                                  
                                                      classes=None, batch_size=1,
                                                      shuffle=False,
                                                      target_size=(h,w),
                                                      interpolation='bilinear',
                                                      seed=seed)
test6 = test_image_data_generator.flow_from_directory(
                                                      '/content/drive/MyDrive/challenge2/Development_Dataset/Test_Dev/Weedelec/Haricot',                                    
                                                      classes=None, batch_size=1,
                                                      shuffle=False,
                                                      interpolation='bilinear',
                                                      target_size=(h,w),
                                                      seed=seed)
test7 = test_image_data_generator.flow_from_directory(
                                                      '/content/drive/MyDrive/challenge2/Development_Dataset/Test_Dev/Weedelec/Mais',                                    
                                                      classes=None, batch_size=1,
                                                      shuffle=False,
                                                      target_size=(h,w),
                                                      interpolation='bilinear',
                                                      seed=seed)

Found 15 images belonging to 1 classes.
Found 15 images belonging to 1 classes.
Found 15 images belonging to 1 classes.
Found 15 images belonging to 1 classes.
Found 15 images belonging to 1 classes.
Found 15 images belonging to 1 classes.
Found 15 images belonging to 1 classes.
Found 15 images belonging to 1 classes.


In [ ]:

#Take list of test generators and returns their names 
def names_from_test_generator(generator:list):
  list_result = []
  for gen in generator:
    result = []
    for elem in gen.filenames:
      result.append(elem.split('/')[1].split('.')[0])
    list_result.append(result)
  return list_result

In [ ]:
#Take the list of test generator and return the predictions
#We could do the upsampling after the model.predict
def predictions_from_generator(generator:list, model):
  list_result = []
  for gen in generator:
    result = []
    predictions = model.predict(gen, batch_size=1)
    for i in range(predictions.shape[0]):
      result.append(predictions[i])
    list_result.append(result)
  return list_result

Create list of test-generator. Get the names and the respective predictions

In [ ]:
test_list = [test, test1, test2, test3, test4, test5, test6, test7] #All the test generators of the different classes
names = names_from_test_generator(test_list) #Get names of the images
predictions = predictions_from_generator(test_list, model) #Assuming model is your trained model. Get prediction of the images

The encode function was provided by the starting kit of the challenge. We adapted it for our custom dataset

In [ ]:
import os
import json
import numpy as np
from PIL import Image
import cv2



def rle_encode(img):
    '''
    img: numpy array, 1 - foreground, 0 - background
    Returns run length as string formatted
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

#Function to create the json file 
#The cv2 function inverts height with width :(
def submit(names:list, predictions:list):

    submission_dict = {}
    for img_name, mask_arr in zip(names[0], predictions[0]):

      f = tf.argmax(mask_arr, -1)
      mask_arr = np.array(f)
      mask_arr = cv2.resize(mask_arr, dsize=(2048, 1536), interpolation=cv2.INTER_NEAREST) #The cv2 function inverts height with width :(

      submission_dict[img_name] = {}
      submission_dict[img_name]['shape'] = mask_arr.shape
      submission_dict[img_name]['team'] = 'Bipbip'
      submission_dict[img_name]['crop'] = 'Haricot'
      submission_dict[img_name]['segmentation'] = {}

      # RLE encoding
      # crop
      rle_encoded_crop = rle_encode(mask_arr == 1)
      # weed
      rle_encoded_weed = rle_encode(mask_arr == 2)

      submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
      submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed
    
    for img_name, mask_arr in zip(names[1], predictions[1]):
      f = tf.argmax(mask_arr, -1)
      mask_arr = np.array(f)
      mask_arr = cv2.resize(mask_arr, dsize=(2048, 1536), interpolation=cv2.INTER_NEAREST)

      submission_dict[img_name] = {}
      submission_dict[img_name]['shape'] = mask_arr.shape
      submission_dict[img_name]['team'] = 'Bipbip'
      submission_dict[img_name]['crop'] = 'Mais'
      submission_dict[img_name]['segmentation'] = {}

      # RLE encoding
      # crop
      rle_encoded_crop = rle_encode(mask_arr == 1)
      # weed
      rle_encoded_weed = rle_encode(mask_arr == 2)

      submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
      submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed

    for img_name, mask_arr in zip(names[2], predictions[2]):
      f = tf.argmax(mask_arr, -1)
      mask_arr = np.array(f)
      mask_arr = cv2.resize(mask_arr, dsize=(3280, 2464), interpolation=cv2.INTER_NEAREST)

      submission_dict[img_name] = {}
      submission_dict[img_name]['shape'] = mask_arr.shape
      submission_dict[img_name]['team'] = 'Pead'
      submission_dict[img_name]['crop'] = 'Haricot'
      submission_dict[img_name]['segmentation'] = {}

      # RLE encoding
      # crop
      rle_encoded_crop = rle_encode(mask_arr == 1)
      # weed
      rle_encoded_weed = rle_encode(mask_arr == 2)

      submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
      submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed

    for img_name, mask_arr in zip(names[3], predictions[3]):
      f = tf.argmax(mask_arr, -1)
      mask_arr = np.array(f)
      mask_arr = cv2.resize(mask_arr, dsize=(3280, 2464), interpolation=cv2.INTER_NEAREST)

      submission_dict[img_name] = {}
      submission_dict[img_name]['shape'] = mask_arr.shape
      submission_dict[img_name]['team'] = 'Pead'
      submission_dict[img_name]['crop'] = 'Mais'
      submission_dict[img_name]['segmentation'] = {}

      # RLE encoding
      # crop
      rle_encoded_crop = rle_encode(mask_arr == 1)
      # weed
      rle_encoded_weed = rle_encode(mask_arr == 2)

      submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
      submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed

    for img_name, mask_arr in zip(names[4], predictions[4]):
      f = tf.argmax(mask_arr, -1)
      mask_arr = np.array(f)
      mask_arr = cv2.resize(mask_arr, dsize=(1227, 819), interpolation=cv2.INTER_NEAREST)

      submission_dict[img_name] = {}
      submission_dict[img_name]['shape'] = mask_arr.shape
      submission_dict[img_name]['team'] = 'Roseau'
      submission_dict[img_name]['crop'] = 'Haricot'
      submission_dict[img_name]['segmentation'] = {}

      # RLE encoding
      # crop
      rle_encoded_crop = rle_encode(mask_arr == 1)
      # weed
      rle_encoded_weed = rle_encode(mask_arr == 2)

      submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
      submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed

    for img_name, mask_arr in zip(names[5], predictions[5]):
      f = tf.argmax(mask_arr, -1)
      mask_arr = np.array(f)
      mask_arr = cv2.resize(mask_arr, dsize=(1227, 819), interpolation=cv2.INTER_NEAREST)

      submission_dict[img_name] = {}
      submission_dict[img_name]['shape'] = mask_arr.shape
      submission_dict[img_name]['team'] = 'Roseau'
      submission_dict[img_name]['crop'] = 'Mais'
      submission_dict[img_name]['segmentation'] = {}

      # RLE encoding
      # crop
      rle_encoded_crop = rle_encode(mask_arr == 1)
      # weed
      rle_encoded_weed = rle_encode(mask_arr == 2)

      submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
      submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed

    for img_name, mask_arr in zip(names[6], predictions[6]):
      f = tf.argmax(mask_arr, -1)
      mask_arr = np.array(f)
      mask_arr = cv2.resize(mask_arr, dsize=(5184, 3456), interpolation=cv2.INTER_NEAREST)

      submission_dict[img_name] = {}
      submission_dict[img_name]['shape'] = mask_arr.shape
      submission_dict[img_name]['team'] = 'Weedelec'
      submission_dict[img_name]['crop'] = 'Haricot'
      submission_dict[img_name]['segmentation'] = {}

      # RLE encoding
      # crop
      rle_encoded_crop = rle_encode(mask_arr == 1)
      # weed
      rle_encoded_weed = rle_encode(mask_arr == 2)

      submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
      submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed

    for img_name, mask_arr in zip(names[7], predictions[7]):
      f = tf.argmax(mask_arr, -1)
      mask_arr = np.array(f)
      mask_arr = cv2.resize(mask_arr, dsize=(5184, 3456), interpolation=cv2.INTER_NEAREST)

      submission_dict[img_name] = {}
      submission_dict[img_name]['shape'] = mask_arr.shape
      submission_dict[img_name]['team'] = 'Weedelec'
      submission_dict[img_name]['crop'] = 'Mais'
      submission_dict[img_name]['segmentation'] = {}

      # RLE encoding
      # crop
      rle_encoded_crop = rle_encode(mask_arr == 1)
      # weed
      rle_encoded_weed = rle_encode(mask_arr == 2)

      submission_dict[img_name]['segmentation']['crop'] = rle_encoded_crop
      submission_dict[img_name]['segmentation']['weed'] = rle_encoded_weed
    
    with open('submission.json', 'w') as f:
        json.dump(submission_dict, f)

In [ ]:
submit(names, predictions)